## Вопросы для самопроверки:
* что такое обучени с подкреплением (reinforcement learning)?
* что такое среда?
* что такое агент?
* что такое награда, какая она может быть?

In [6]:
import gym
import numpy as np

#create a single game instance
env = gym.make("Taxi-v2")

#start new game
env.reset();

[2017-10-24 20:40:56,231] Making new env: Taxi-v2


In [7]:
# display the game state
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [8]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 454
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

observations: Discrete(500) n= 500
actions: Discrete(6) n= 6


In [9]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 454
reward: -1
is game over?: False
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [10]:
action_to_i = {
    'left':3,
    'down':0,
    'right':2,
    'up':1,
    'pick':4,
    'drop':5,
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [11]:
env.step(action_to_i['down'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


### Baseline: random search

In [12]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n

def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.choice(replace=True,a=np.arange(6),size=500)

In [13]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.1668264  0.1667818  0.166578   0.166587   0.166508   0.1667188]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [14]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    reward = 0
    cur_state = 0
    
    for _ in np.arange(t_max):
        s, reward, is_done, _ = env.step(policy[s])
        if is_done:
            break
    return reward

In [15]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
# assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake (if solving taxi, delete this line)'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [16]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

In [17]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way."""
    field = "+---------+\
            |R: | : :G|\
            | : : : : |\
            | : : : : |\
            | | : | : |\
            |Y| : |B: |\
            +---------+"
    #assert env.spec.id == "FrozenLake-v0", "this function only works with frozenlake 4x4"

    
    # where to move from each tile (we're a bit unsure if this is accurate)
    arrows = ['>^v<pd'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile not in "|+-:" else tile for arrow, tile in zip(arrows, field)]
    
    for i in range(0, 100, 10):
        print(' '.join(signs[i:i+10]))

print("random policy:")
print_policy(get_random_policy())

random policy:
+ - - - - - - - - -
+ < < p < d ^ p v >
p p d | p : p | > :
^ : d | > ^ < d ^ >
d < < p p > | ^ : v
: p : p : < | ^ < v
< < d d p < ^ d v |
< : > : > : d : > |
> v < d > > d < ^ <
v > | ^ | d : p | v


### Main loop

In [53]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(500), position=0):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
#         print("Best policy:")
#         print_policy(best_policy)

  0%|          | 2/500 [00:00<01:08,  7.28it/s]

New best score: -7.12
New best score: -4.87


 18%|█▊        | 91/500 [00:11<00:50,  8.05it/s]

New best score: -4.42


 92%|█████████▏| 459/500 [00:57<00:05,  8.02it/s]

New best score: -4.33


100%|██████████| 500/500 [01:03<00:00,  8.07it/s]


# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [54]:
import scipy.stats as sps
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    choose = sps.bernoulli.rvs(size=policy1.shape, p=p)
    return np.where(choose, policy1, policy2)

In [55]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(get_random_policy(), policy, p)
    

In [56]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [4]:
n_epochs = 200 #how many cycles to make
pool_size = 150 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick

In [5]:
print("initializing...")
pool = [get_random_policy() for _ in np.arange(pool_size)]
pool_scores = [evaluate(policy) for policy in pool]

initializing...


NameError: name 'np' is not defined

In [69]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])


In [2]:
#main loop
modify_for_tel = lambda s: "'" + s + "'"
config = "'Start Taxi Learning with n_epoches: {}, pool_size: {}, n_crossovers: {}, n_mutations: {}'".format(
    n_epochs, pool_size, n_crossovers, n_mutations,
)
print(config)
config = modify_for_tel(config)
!telegram-send $config
for epoch in range(n_epochs):
    epoch_start_msg = "Epoch %s:"%epoch
    print(epoch_start_msg)
    
    !telegram-send $epoch_start_msg
    crossovered = [crossover(pool[ind[0]], pool[ind[1]]) 
                   for ind in np.random.choice(a=np.arange(len(pool)), size=2*n_crossovers).reshape((-1, 2))]
    mutated = [mutation(pool[ind]) for ind in np.random.choice(a=np.arange(len(pool)),size=n_mutations)]
    
    #add new policies to the pool
    pool.extend(crossovered)
    pool.extend(mutated)
    pool_scores = [evaluate(policy) for policy in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    line = "best score: {}".format(pool_scores[-1]) 
    print(line)
    line = modify_for_tel(line)
    !telegram-send $line

NameError: name 'n_epochs' is not defined